# Note: this will not work on Jetson because pip3 opencv does not support reading video frames from file! Use the desktop instead.

In [ ]:
import numpy as np
from scipy import interpolate
import os
import PIL.Image as Image
import cv2
import pickle as pkl
import pyzed.sl as sl
import bisect

In [ ]:
DIR = '/home/nvidia/myrecordings/'
CSI_VIDEO = 'CSI_2020-09-30-11-14-18.avi'
CSI_PKL = 'CSI_2020-09-30-11-14-18.pkl'
ZED_SVO = 'ZED_2020-09-30-11-14-14.svo'

CSI_VIDEO_PATH = os.path.join(DIR, CSI_VIDEO)
CSI_PKL_PATH = os.path.join(DIR, CSI_PKL)
ZED_SVO_PATH = os.path.join(DIR, ZED_SVO)

In [ ]:
vidcap = cv2.VideoCapture(CSI_VIDEO_PATH)
vidcap.isOpened()
success, img = vidcap.read()

In [ ]:
with open(CSI_PKL_PATH, 'rb') as f:
    csi_timestamps = np.array(pkl.load(f))

In [ ]:
init_parameters = sl.InitParameters()
init_parameters.set_from_svo_file(ZED_SVO_PATH)

zed = sl.Camera()
err = zed.open(init_parameters)
runtime = sl.RuntimeParameters()
svo_image = sl.Mat()
depth_map = sl.Mat()

if zed.grab() == sl.ERROR_CODE.SUCCESS:
    zed.retrieve_image(svo_image, sl.VIEW.DEPTH)
    zed.retrieve_measure(depth_map, sl.MEASURE.DEPTH, sl.MEM.CPU)
    svo_position = zed.get_svo_position()
    t = zed.get_timestamp(sl.TIME_REFERENCE.IMAGE).get_nanoseconds()/1e9
    # finds the closest frame, usually correct to a few milliseconds
    frame_idx = bisect.bisect_left(csi_timestamps, t)
    time_diff = abs(csi_timestamps[frame_idx]-t)
    if vidcap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx):
        success, csi_img = vidcap.read()
    # Now do your processing on csi_img, svo_image, and depth_map